# Download weight

In [17]:
# Delete all files and folders in the current directory. Use with caution!
import shutil
shutil.rmtree('./', ignore_errors=True)

In [18]:
!git clone https://github.com/BinhPQ2/Han_Nom_OCR.git .
!git clone https://github.com/BinhPQ2/yolov10.git

Cloning into '.'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 228 (delta 55), reused 204 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (228/228), 8.89 MiB | 34.08 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Cloning into 'yolov10'...
remote: Enumerating objects: 20328, done.
remote: Counting objects: 100% (2443/2443), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 20328 (delta 2315), reused 2198 (delta 2198), pack-reused 17885 (from 1)
Receiving objects: 100% (20328/20328), 11.14 MiB | 24.33 MiB/s, done.
Resolving deltas: 100% (14335/14335), done.


In [19]:
!gdown --id weight_link_here -O file.zip
!unzip -q file.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w
From (redirected): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w&confirm=t&uuid=ab853c6f-df67-4f24-a2dd-9f431a3bf79f
To: /content/file.zip
100% 262M/262M [00:01<00:00, 245MB/s]


# Install environments

In [3]:
!pip install -q paddleocr --upgrade
!pip install -q paddlepaddle-gpu --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 809.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install -q -r requirements.txt
!pip install -q -e ./yolov10/
# !pip install supervision -q # already included in the requirement.txt
# !pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/

In [ ]:
!pip show ultralytics

Name: ultralytics
Version: 8.1.34
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Editable project location: /content/yolov10
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 


In [4]:
import os
import random
import numpy as np
from tqdm import tqdm
import pickle
import cv2
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLOv10 # rerun session if it has error in this line

# Detection

In [ ]:
def save_crop(result, save_path):
    os.makedirs(save_path, exist_ok=True)

    conf_s = result.boxes.conf.detach().cpu().numpy()
    xyxy_s = result.boxes.xyxy.detach().cpu().numpy()
    idx_s = [i for i in range(len(conf_s))]
    img_name, img_extension = os.path.splitext(os.path.basename(result.path))
    img = result.orig_img

    height_extend_ratio = 0.05
    width_extend_ratio = 0.1
    height=result.orig_img.shape[0]
    width=result.orig_img.shape[1]

    for idx, conf, xyxy in zip(idx_s, conf_s, xyxy_s):
        # name = f"oder_{idx}__conf_{conf}__name_{img_name}"

        x0, y0, x1, y1 = xyxy.tolist()
        x0 = int(max(0, x0 - width_extend_ratio*abs(x1 - x0)))
        x1 = int(min(width, x1 + width_extend_ratio*abs(x1 - x0)))
        y0 = int(max(0, y0 - height_extend_ratio*abs(y1 - y0)))
        y1 = int(min(height, y1 + height_extend_ratio*abs(y1 - y0)))
        crop_image = img[y0:y1, x0:x1]
        name = f"{img_name}_{x0}{img_extension}"


        cv2.imwrite(os.path.join(save_path, name), crop_image)

In [ ]:
input_image_folder = "./input_image"
save_path = "./output_detection"
model = YOLOv10('./weight/detection_yolov10.pt')

results = model.predict(source=input_image_folder, conf=0.5, stream=True, save_txt=True)
for idx, result in enumerate(results):
    save_crop(result, save_path)


image 1/23 /content/input_image/02b02efc5d6b4579a0cae31e1e362555.jpg: 640x480 1 couplet, 64.5ms
image 2/23 /content/input_image/0ff04276ca994b12a50f51f88a9373fc.jpg: 640x352 1 couplet, 67.1ms
image 3/23 /content/input_image/1a2291615b4e4c288a152b4886ac300e.jpg: 640x480 1 couplet, 30.1ms
image 4/23 /content/input_image/1a5634d0281b4b1aac6e616826c30d4b.jpg: 640x320 2 couplets, 165.6ms
image 5/23 /content/input_image/1a7296dbb22748e49b656df5cc6743b5.jpg: 640x288 2 couplets, 66.4ms
image 6/23 /content/input_image/1b094caec3984b8eb1c929b60a8d2b08.jpg: 640x480 4 couplets, 30.3ms
image 7/23 /content/input_image/1b648fd456b54ea5a6a4ff1222b95bad.jpg: 640x384 1 couplet, 63.9ms
image 8/23 /content/input_image/1b7b10992eb040d19fcfcadb745d1aa0.jpg: 640x256 1 couplet, 62.0ms
image 9/23 /content/input_image/1be6c13d483c41e6809a843fb9f1e913.jpg: 640x480 1 couplet, 30.0ms
image 10/23 /content/input_image/1bf527e50d2447098ef82c510e5dab4d.jpg: 640x288 1 couplet, 23.7ms
image 11/23 /content/input_image/1

# Rotate image to prepare for Recognition

In [ ]:
# prompt: Rotate all the image in ./output_detection by 90 degree counter-clockwise and save it in ./output_detection_rotated

import os
import cv2

input_folder = './output_detection'
output_folder = './output_detection_rotated'

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
  if filename.endswith(('.jpg', '.jpeg', '.png')):
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)
    rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, rotated_img)


# Recognition

In [ ]:
!python3 /content/recognition/ppocr/tools/infer/predict_rec.py --image_dir="./output_detection_rotated" \
                                                    --rec_model_dir='./weight/recognition_PPOCRLABEL' \
                                                    --rec_char_dict_path='/content/recognition/ppocr/utils/ppocr_keys_v1.txt' \
                                                    # --use_gpu=False \

[2024/08/31 16:30:38] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/08/31 16:30:41] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2024/08/31 16:30:43] ppocr INFO: Predicts of ./output_detection_rotated/02b02efc5d6b4579a0cae31e1e362555.jpg:('光飛以安邦寶劍秋藏湯色近', 0.8345915675163269)
[2024/08/31 16:30:43] ppocr INFO: Predicts of ./output_detection_rotated/0ff04276ca994b12a50f51f88a9373fc.jpg:('天太平文官不爱钱官不惜死', 0.8724350929260254)
[2024/08/31 16:30:43] ppocr INFO: Predicts of ./output_detection_rotated/1a2291615b4e4c288a152b4886ac300e.jpg:('鴻飛歸結以來永傳留國祖', 0.8624740839004517)
[2024/08/31 16:30:43] ppocr INFO: Predicts of ./output_detection_rotated/1a5634d0281b4b1aac6e616826c30d4b.jpg:('家慶有裕波同', 0.9257786273956299)
[2024/08/31 16:30:43] ppocr INFO: Predicts of ./output_detection_rotated/1a7296dbb22748e49b656df5cc6743b5.jpg:('延年收厚

# Testing module usage

In [ ]:
class Args:
    def __init__(self, **kwargs):
        # Set attributes dynamically based on keyword arguments
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self):
        # Optional: customize the string representation for debugging
        return str(self.__dict__)

# Usage example:
args = Args(
    use_gpu=True,
    use_xpu=False,
    use_npu=False,
    use_mlu=False,
    ir_optim=True,
    use_tensorrt=False,
    min_subgraph_size=15,
    precision='fp32',
    gpu_mem=500,
    gpu_id=0,
    image_dir='./output_detection_rotated',
    page_num=0,
    det_algorithm='DB',
    det_model_dir=None,
    det_limit_side_len=960,
    det_limit_type='max',
    det_box_type='quad',
    det_db_thresh=0.3,
    det_db_box_thresh=0.6,
    det_db_unclip_ratio=1.5,
    max_batch_size=10,
    use_dilation=False,
    det_db_score_mode='fast',
    det_east_score_thresh=0.8,
    det_east_cover_thresh=0.1,
    det_east_nms_thresh=0.2,
    det_sast_score_thresh=0.5,
    det_sast_nms_thresh=0.2,
    det_pse_thresh=0,
    det_pse_box_thresh=0.85,
    det_pse_min_area=16,
    det_pse_scale=1,
    scales=[8, 16, 32],
    alpha=1.0,
    beta=1.0,
    fourier_degree=5,
    rec_algorithm='SVTR_LCNet',
    rec_model_dir='./weight/recognition_PPOCRLABEL',
    rec_image_inverse=True,
    rec_image_shape='3, 48, 320',
    rec_batch_num=6,
    max_text_length=25,
    rec_char_dict_path='/content/recognition/ppocr/utils/ppocr_keys_v1.txt',
    use_space_char=True,
    vis_font_path='./doc/fonts/simfang.ttf',
    drop_score=0.5,
    e2e_algorithm='PGNet',
    e2e_model_dir=None,
    e2e_limit_side_len=768,
    e2e_limit_type='max',
    e2e_pgnet_score_thresh=0.5,
    e2e_char_dict_path='./ppocr/utils/ic15_dict.txt',
    e2e_pgnet_valid_set='totaltext',
    e2e_pgnet_mode='fast',
    use_angle_cls=False,
    cls_model_dir=None,
    cls_image_shape='3, 48, 192',
    label_list=['0', '180'],
    cls_batch_num=6,
    cls_thresh=0.9,
    enable_mkldnn=False,
    cpu_threads=10,
    use_pdserving=False,
    warmup=False,
    sr_model_dir=None,
    sr_image_shape='3, 32, 128',
    sr_batch_num=1,
    draw_img_save_dir='./inference_results',
    save_crop_res=False,
    crop_res_save_dir='./output',
    use_mp=False,
    total_process_num=1,
    process_id=0,
    benchmark=False,
    save_log_path='./log_output/',
    show_log=True,
    use_onnx=False,
    return_word_box=False
)

from recognition.ppocr.tools.infer.predict_mine import main
main(args)

[2024/08/31 16:30:59] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/08/31 16:31:02] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2024/08/31 16:31:02] ppocr INFO: Predicts of ./output_detection_rotated/02b02efc5d6b4579a0cae31e1e362555.jpg:('光飛以安邦寶劍秋藏湯色近', 0.8345918655395508)
[2024/08/31 16:31:02] ppocr INFO: Predicts of ./output_detection_rotated/0ff04276ca994b12a50f51f88a9373fc.jpg:('天太平文官不爱钱官不惜死', 0.872434675693512)
[2024/08/31 16:31:02] ppocr INFO: Predicts of ./output_detection_rotated/1a2291615b4e4c288a152b4886ac300e.jpg:('鴻飛歸結以來永傳留國祖', 0.8624739050865173)
[2024/08/31 16:31:02] ppocr INFO: Predicts of ./output_detection_rotated/1a5634d0281b4b1aac6e616826c30d4b.jpg:('家慶有裕波同', 0.925778865814209)
[2024/08/31 16:31:02] ppocr INFO: Predicts of ./output_detection_rotated/1a7296dbb22748e49b656df5cc6743b5.jpg:('延年收厚',

# Running in just 1 line using CLI

In [ ]:
!python3 main.py


image 1/23 /content/input_image/02b02efc5d6b4579a0cae31e1e362555.jpg: 640x480 1 couplet, 69.1ms
image 2/23 /content/input_image/0ff04276ca994b12a50f51f88a9373fc.jpg: 640x352 1 couplet, 78.2ms
image 3/23 /content/input_image/1a2291615b4e4c288a152b4886ac300e.jpg: 640x480 1 couplet, 30.1ms
image 4/23 /content/input_image/1a5634d0281b4b1aac6e616826c30d4b.jpg: 640x320 2 couplets, 65.2ms
image 5/23 /content/input_image/1a7296dbb22748e49b656df5cc6743b5.jpg: 640x288 2 couplets, 85.2ms
image 6/23 /content/input_image/1b094caec3984b8eb1c929b60a8d2b08.jpg: 640x480 4 couplets, 30.0ms
image 7/23 /content/input_image/1b648fd456b54ea5a6a4ff1222b95bad.jpg: 640x384 1 couplet, 65.5ms
image 8/23 /content/input_image/1b7b10992eb040d19fcfcadb745d1aa0.jpg: 640x256 1 couplet, 74.9ms
image 9/23 /content/input_image/1be6c13d483c41e6809a843fb9f1e913.jpg: 640x480 1 couplet, 30.0ms
image 10/23 /content/input_image/1bf527e50d2447098ef82c510e5dab4d.jpg: 640x288 1 couplet, 23.6ms
image 11/23 /content/input_image/1c

In [ ]:
!rm -rf ./output_detection_rotated
!rm -rf ./output_detection

# Running using streamlit
Tutorial: https://www.youtube.com/watch?v=CzG1_S8mcFc

In [5]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [6]:
! wget -q -O - ipv4.icanhazip.com # paste the port below into the box when run the command below

34.105.35.179


In [ ]:
! streamlit run app_streamlit.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.35.179:8501

your url is: https://tricky-dots-yawn.loca.lt

image 1/1 /tmp/tmp4kdds8nz/tmpwev0uiag.jpg: 640x640 2 couplets, 35.7ms
Speed: 3.3ms preprocess, 35.7ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/yolov10/runs/detect/predict
1 label saved to /content/yolov10/runs/detect/predict/labels
[2024/09/03 11:14:38] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/09/03 11:14:40] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2024/09/03 11:14:40] ppocr INFO: Predicts of /tmp/tmpj2y6aw9h/tmpwev0uiag_1440.jpg:('雲潤流清磨', 0.7589268684387207)
[2024/09/03 11:14:40] ppocr INFO: Predicts of /tmp/tmpj2y6aw9h/tmpwev0uiag_21